In [ ]:
import pandas as pd
import altair as alt
import datetime
import sqlite3
from mlrepricer import helper, setup, schemas, listener
from mlrepricer.database import SQLite
%alias_magic t timeit
import os

alt.data_transformers.enable('default', max_rows=1000000)

# listener demon
to get messages from mws queues and dump them into a filesystem

In [ ]:
listener.Listener().start()

### load the results

In [ ]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

### example analysis

In [ ]:
sub = df[(df.asin=='B01H0GZ2M2') & (df.isbuyboxwinner==1)]
sub.index = sub.time_changed
primesub = sub[sub.isprime==1]
primesub.sort_index(inplace=True)
primesub.loc[primesub.index.duplicated(keep=False)]

In [ ]:
# that's a helpful pseudo multiindex, it's superior since we save a colomn and it's speaking
df['messageid'] = df['asin']+'_'+df['time_changed'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
latestdf = df.groupby('asin')['time_changed'].max()
lookup_snapshot = latestdf.index+'_'+latestdf.dt.strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
len(df.groupby('messageid').max())

In [ ]:
snapshot = df.loc[df['messageid'].isin(lookup_snapshot)]

In [ ]:
snapshot.head()

In [ ]:
sortedgroup = df.groupby(['asin'])['time_changed'].count().sort_values(ascending=False)
asins = list(sortedgroup.index)[0:10]
asin = [list(sortedgroup.index)[0]]

In [ ]:
asins

In [ ]:
examplelistings = df[df['asin'].isin(asins)]
examplelisting = df[df['asin'].isin(asin)]

In [ ]:
a = alt.Chart(examplelisting).mark_line(opacity=0.9).encode(
    alt.X('time_changed'),
    y='price',
    color='sellerid:N').interactive()

In [ ]:
a.savechart('chart.svg')

In [ ]:
alt.Chart(examplelistings).mark_line().encode(
    x='time_changed:T',
    y='min(price)',
    color='asin')

In [ ]:
winner = examplelistings[examplelistings['isbuyboxwinner'] == 1]

In [ ]:
alt.Chart(winner).mark_line().encode(
    x='time_changed:T',
    y='min(price)',
    color='asin')